In [2]:
'''
一个子类（派生类）可以继承多个父类（基类）

subclass.__base__  # 查看从左到右继承的第一个父类（元组）
subclass.__bases__ # 查看所有继承的父类（元组）

'''
# EG1. 属性查找顺序：先是本实例，然后是父类
class Foo():
    def f1(self):
        print('from Foo.f1')
    def f2(self):
        print('from Foo.f2')
        self.f1()
        
class Bar(Foo):
    def f1(self):
        print('from Bar.f1')

b = Bar()
b.f2()

# 区别于上篇“1.OOP：封装”中，封装的私有方法篇

from Foo.f2
from Bar.f1


In [4]:
# EG2. 继承顺序
class A:
    def test(self):
        print('from A')
        
class B(A):
    def test(self):
        print('from B')
        
class C(A):
    def test(self):
        print('from C')
        
class D(B):
    def test(self):
        print('from D')
        
class E(C):
    def test(self):
        print('from E')
        
class F(D,E):
    def test(self):
        print('from F')
        
f = F()
f.test()
print(F.__mro__) # 只有新式类才有这个属性查看线性列表，经典类没有；实例化的对象也没有

from F
(<class '__main__.F'>, <class '__main__.D'>, <class '__main__.B'>, <class '__main__.E'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


In [8]:
# EG3. 类组合：类之间的相互调用，当类之间有显著不同，并且较小的类是较大的类所需要的组件时，用组合比较好
class People:
    def __init__(self, name, age, sex):
        self.name = name
        self.age = age
        self.sex = sex
        
class Teacher(People):
    def __init__(self, name, age, sex, job_title):
        People.__init__(self, name, age, sex)
        self.job_title = job_title
        self.courses = []
        self.students = []
        
class Course:
    def __init__(self, name, period, price):
        self.name = name
        self.period = period
        self.price = price
        
    def tell_info(self):
        print('<%s %s %s>' %(self.name, self.period, self.price))
        
class Student(People):
    def __init__(self, name, age, sex):
        People.__init__(self, name, age, sex)
        self.courses = []

# 实例化对象
egon = Teacher('egon', 18, 'female', '沙河霸道金牌讲师')
s1 = Student('牛榴弹',18,'female')

python = Course('python','3mons',3000)
linux = Course('linux','5mons',5000)

# 为老师和学生添加课程(实例对象)
egon.courses.append(python)
egon.courses.append(linux)
s1.courses.append(python)

# 为老师添加学生
egon.students.append(s1)

print(egon.courses)
print(s1.courses)
for obj in egon.courses:
    obj.tell_info()
    

[<__main__.Course object at 0x000001E60EC204C0>, <__main__.Course object at 0x000001E60E9757F0>]
<python 3mons 3000>
<linux 5mons 5000>


In [12]:
'''EG4. 子类中，调用父类方法'''
# _*_ 方法1.指名道姓即 父类名.父类方法() _*_ #
class Vehicle:
    def __init__(self, name, speed, load, power):
        self.name, self.speed, self.load, self.power = name, speed, load, power
    
    def run(self):
        print('Going...')
        
class Subway(Vehicle):
    def __init__(self, name, speed, load, power, line):
        Vehicle.__init__(self, name, speed, load, power)
        self.line = line
        
    def run(self):
        print('地铁%s号线，欢迎你' % self.line)
        Vehicle.run(self)
        
line13=Subway('中国地铁','180m/s','1000人/箱','电',13)
line13.run()

# _*_ 方法2.super() _*_ #
class Mobike(Vehicle):
    country = 'China'
    
    def __init__(self, name, speed, load, power):
        # super() == super(Mobike, self) == Vehicle(self), 即 super() 自带参数self 
        super().__init__(name, speed, load, power)
    
    def run(self):
        print('Hello, welcome to using Mobike!')
        super().run()
        
mb = Mobike('摩拜', '30m/h', 1, 'manpower')
mb.run()

地铁13号线，欢迎你
Going...
Hello, welcome to using Mobike!
Going...


In [17]:
# 两种调用父类方法的区别
'''
当你使用super()函数时,Python会在MRO列表上继续搜索下一个类。
只要每个重定义的方法统一使用super()并只调用它一次,那么控制流最终会遍历完整个MRO列表,每个方法也只会被调用一次
（注意：使用super调用的所有属性，都是从MRO列表当前的位置往后找，千万不要通过看代码去找继承关系，一定要看MRO列表）
'''
# 1.
class A:
    def __init__(self):
        print('A func')
        
class B(A):
    def __init__(self):
        print('B func')
        A.__init__(self)

class C(A):
    def __init__(self):
        print('C func')
        A.__init__(self)
        
class D(B,C):
    def __init__(self):
        print('D func')
        B.__init__(self)
        C.__init__(self)
print(D.__mro__)
d = D() 

# 2.
class A:
    def __init__(self):
        print('A func')
        
class B(A):
    def __init__(self):
        print('B func')
        super().__init__()

class C(A):
    def __init__(self):
        print('C func')
        super().__init__()
        
class D(B,C):
    def __init__(self):
        print('D func')
        super().__init__()
print(D.__mro__)        
d = D()

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
D func
B func
A func
C func
A func
(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
D func
B func
C func
A func


In [ ]:
# _*_ EG5. 接口与归一化设计 _*_ #
'''
接口：把实现某个功能的一组类包装成函数集合，然后让子类继承基类，并实现接口中的函数。

意义在于归一化：只要是基于同一个接口实现的类，那么所有的这些类产生的对象在使用时，从用法上来说都一样。
'''

# _*_ EG6. 抽象类 _*_ #
'''
抽象类是一个特殊的类，只能被继承，不能被实例化

类是对（一堆对象）的抽象，抽象类是对（一堆类）的抽象：
    从设计角度去看，如果类是从现实对象抽象而来的，那么抽象类就是基于类抽象而来的。
    从实现角度来看，抽象类中只能有抽象方法（没有实现功能），该类不能被实例化，只能被继承，且子类必须实现抽象方法。

抽象类的本质还是类，指的是一组类的相似性，包括数据属性（如all_type）和函数属性（如read ,write），
而接口只强调函数属性的相似性
'''
import abc #利用abc模块实现抽象类
 
class All_file(metaclass=abc.ABCMeta):
    all_type='file'
    @abc.abstractmethod #定义抽象方法，无需实现功能
    def read(self):
        '子类必须定义读功能'
        pass
 
    @abc.abstractmethod #定义抽象方法，无需实现功能
    def write(self):
        '子类必须定义写功能'
        pass
 
class Txt(All_file): #子类继承抽象类，但是必须定义read和write方法
    def read(self):
        print('文本数据的读取方法')
 
    def write(self):
        print('文本数据的读取方法')
 
class Sata(All_file): #子类继承抽象类，但是必须定义read和write方法
    def read(self):
        print('硬盘数据的读取方法')
 
    def write(self):
        print('硬盘数据的读取方法')
 
class Process(All_file): #子类继承抽象类，但是必须定义read和write方法
    def read(self):
        print('进程数据的读取方法')
 
    def write(self):
        print('进程数据的读取方法')
 
wenbenwenjian=Txt()
 
yingpanwenjian=Sata()
 
jinchengwenjian=Process()
 
#这样大家都是被归一化了,也就是一切皆文件的思想
wenbenwenjian.read()
yingpanwenjian.write()
jinchengwenjian.read()
 
print(wenbenwenjian.all_type)
print(yingpanwenjian.all_type)
print(jinchengwenjian.all_type)